In [6]:
############################################################
#
#不使用impute
#####依赖higashi使用原始的稀疏矩阵计算sc和prebulk的pc值
##########################################################
#############################################################
#############################################################
##################################################################
##################################################################
from Higashi_backend.utils import *
from Higashi_analysis.Higashi_analysis import *
import h5py
from sklearn.preprocessing import MinMaxScaler, quantile_transform
import os
import numpy as np
os.environ["OMP_NUM_THREADS"] = "10"
import pandas as pd
import argparse
import pickle
import scipy







def rankmatch(from_mtx, to_mtx):
	temp = np.sort(to_mtx.reshape((-1)))
	temp2 = from_mtx.reshape((-1))
	order = np.argsort(temp2)
	temp2[order] = temp
	return temp2.reshape((len(from_mtx), -1))


def create_mask(k=30, chrom="chr1", origin_sparse=None):
	final = np.array(np.sum(origin_sparse, axis=0).todense())
	size = origin_sparse[0].shape[-1]
	a = np.zeros((size, size))
	if k > 0:
		for i in range(min(k, len(a))):
			for j in range(len(a) - i):
				a[j, j + i] = 1
				a[j + i, j] = 1
		a = np.ones_like((a)) - a
	
	gap = np.sum(final, axis=-1, keepdims=False) == 0
	if cytoband_path is not None:
		gap_tab = pd.read_table(cytoband_path, sep="\t", header=None)
		gap_tab.columns = ['chrom', 'start', 'end', 'name', 'type']
		
		name = np.array(gap_tab['name'])
		# print (name)
		pqarm = np.array([str(s)[0] for s in name])
		gap_tab['pq_arm'] = pqarm
		gap_tab['length'] = gap_tab['end'] - gap_tab['start']
		summarize = gap_tab.groupby(['chrom', 'pq_arm']).sum().reset_index()
		# print (summarize)
		
		if np.sum(summarize['pq_arm'] == 'p') > 0:
			split_point = \
			np.ceil(np.array(summarize[(summarize['chrom'] == chrom) & (summarize['pq_arm'] == 'p')]['length']) / res)[0]
		else:
			split_point = -1
		
		gap_list = gap_tab[(gap_tab["chrom"] == chrom) & (gap_tab["type"] == "acen")]
		start = np.floor((np.array(gap_list['start'])) / res).astype('int')
		end = np.ceil((np.array(gap_list['end'])) / res).astype('int')
		
		for s, e in zip(start, end):
			a[s:e, :] = 1
			a[:, s:e] = 1
	else:
		split_point = -1
	a[gap, :] = 1
	a[:, gap] = 1
	
	return a, int(split_point)


def process_one_chrom(chrom):
	# Get the raw sparse mtx list
	cell_type_info=pickle.load(open(os.path.join(data_dir, "label_info.pickle"), "rb"))
	cell_type=cell_type_info['cell_type']
	origin_sparse = np.load(os.path.join(raw_dir, "%s_sparse_matrices.npy" % chrom), allow_pickle=True)
	size = origin_sparse[0].shape[0]
	# find centromere & gaps...
	mask, split_point = create_mask((int(1e5)), chrom, origin_sparse)

	bulk1 = np.array(np.sum(origin_sparse, axis=0).todense())
	print(bulk1.shape)
	mask = (np.ones_like(bulk1) - mask)
	bulk1 *= mask

	if "bulk_path" in config:
		import cooler
		c = cooler.Cooler("%s::resolutions/%d" % (config['bulk_path'], config['resolution']))
		bulk2 = np.array(c.matrix(sparse=False, balance=False).fetch(chrom)).astype('float')
		bulk2 *= mask
	else:
		bulk2 = None

	use_rows_all = []

	if split_point >= 20 * 1000000 / res:
		slice_start_list, slice_end_list = [0, split_point], [split_point, len(bulk1)]
	else:
		slice_start_list, slice_end_list = [0], [len(bulk1)]

	bulk_compartment_all = []
	real_bulk_compartment_all = []

	bulk_model_list = []
	bulk_reverse_list = []
	bulk_slice_list = []
	use_rows_list = []
	
	# temp_compartment_list_zscore = []
	# temp_compartment_list_quantile = []

	for slice_start, slice_end in zip(slice_start_list, slice_end_list):
		
		bulk1_slice = bulk1[slice_start:slice_end, :]
		bulk1_slice = bulk1_slice[:, slice_start:slice_end]
		use_rows = np.where(np.sum(bulk1_slice > 0, axis=-1) > 0.01 * len(bulk1_slice))[0]
		if len(use_rows) <= 1:
			print("no reliable bins in slice:", slice_start, slice_end)
			continue
		use_rows_all.append(np.arange(slice_start, slice_end)[use_rows])
		use_rows_list.append(use_rows)
		bulk1_slice = bulk1_slice[use_rows, :]
		bulk1_slice = bulk1_slice[:, use_rows]
		
		bulk_slice_list.append(bulk1_slice)
		bulk_expect = []
		for k in range(len(bulk1_slice)):
			diag = np.diag(bulk1_slice, k)
			bulk_expect.append(np.mean(diag))
		
		
		bulk_compartment, model = test_compartment(bulk1_slice, return_PCA=True)
		
		if bulk2 is not None:
			bulk2_slice = bulk2[slice_start:slice_end, :]
			bulk2_slice = bulk2_slice[:, slice_start:slice_end]
			bulk2_slice = bulk2_slice[use_rows, :]
			bulk2_slice = bulk2_slice[:, use_rows]
			real_bulk_compartment, model = test_compartment(bulk2_slice, return_PCA=True)
		else:
			real_bulk_compartment = None
			
		reverse_flag = False
		bulk_compartment_all.append(bulk_compartment)
		bulk_reverse_list.append(reverse_flag)
		bulk_model_list.append(model)
		real_bulk_compartment_all.append(real_bulk_compartment)
		


		
	temp_compartment_list_all = [[] for i in range(len(use_rows_list))]

	cell_list = trange(len(origin_sparse))
	# print(cell_list)
	temp = np.zeros((size, size))
	for i in cell_list:
		temp *= 0.0
		proba = np.array(origin_sparse[i].todense())
		temp+= proba
		temp = temp + temp.T
		temp *= mask
		
		for j in range(len(use_rows_list)):
			slice_start, slice_end = slice_start_list[j], slice_end_list[j]
			temp_slice = temp[slice_start:slice_end, :]
			temp_slice = temp_slice[:, slice_start:slice_end]
			temp_select = temp_slice[use_rows_list[j], :]
			temp_select = temp_select[:, use_rows_list[j]]
			# temp_select = rankmatch(temp_select, bulk_slice_list[j])
			temp_compartment = test_compartment(temp_select, False, bulk_model_list[j], None)
			if bulk_reverse_list[j]:
				temp_compartment = -1 * temp_compartment
			# temp_compartment_list_all[j].append(temp_compartment.reshape((-1)))
			temp_compartment_list_all[j].append(temp_compartment)
			
	for j in range(len(use_rows_list)):
		temp_compartment_list_all[j] = np.stack(temp_compartment_list_all[j], axis=0)
		

	real_bulk_compartment = np.concatenate(real_bulk_compartment_all, axis=0) if bulk2 is not None else None
	bulk_compartment = np.concatenate(bulk_compartment_all, axis=0)
	temp_compartment_list = np.concatenate(temp_compartment_list_all, axis=-1)
	use_rows = np.concatenate(use_rows_all, axis=0)
	print (chrom, "finished")
	# print("temp_compartment_list shape:",temp_compartment_list.shape)

	##########从原始的接触图计算############################################
	origin_bulk_list_all=[[] for i in range(len(use_rows_list))]
	cell_type_info=pickle.load(open(os.path.join(data_dir, "label_info.pickle"), "rb"))
	cell_type=cell_type_info['cell_type']
	# sum_by_label=[]
	a={}
	b={}
	for j in list(set(cell_type)):
		
		indices = [index for index, value in enumerate(cell_type) if value == j]
		a[j]=indices
		b[j] = np.zeros_like(np.array(origin_sparse[0]))
		for i in a[j]:
				proba = np.array(origin_sparse[i])    
				b[j] +=proba
		temp = np.array(b[j].item().todense())
		# print(temp.shape)
		temp *= mask
		for j in range(len(use_rows_list)):
			slice_start, slice_end = slice_start_list[j], slice_end_list[j]
			temp_slice = temp[slice_start:slice_end, :]
			temp_slice = temp_slice[:, slice_start:slice_end]
			temp_select = temp_slice[use_rows_list[j], :]
			temp_select = temp_select[:, use_rows_list[j]]
			# temp_select = rankmatch(temp_select, bulk_slice_list[j])
			temp_compartment = test_compartment(temp_select, False, bulk_model_list[j], None)
			if bulk_reverse_list[j]:
				temp_compartment = -1 * temp_compartment
			# temp_compartment_list_all[j].append(temp_compartment.reshape((-1)))
			origin_bulk_list_all[j].append(temp_compartment)
        
	for j in range(len(use_rows_list)):
		origin_bulk_list_all[j] = np.stack(origin_bulk_list_all[j], axis=0)
	origin_bulk_list = np.stack(origin_bulk_list_all[j], axis=-1)

	return real_bulk_compartment, bulk_compartment, chrom, use_rows, size,origin_bulk_list,temp_compartment_list

	
			

def process_calib_file(file_path):
	tab = pd.read_table(file_path , sep="\t", header=None)
	tab.columns = ['chrom', 'bin', 'value']
	# print(tab)
	chrom_start_end = np.load(os.path.join(temp_dir, "chrom_start_end.npy"))
	tab['chrom'] = np.array(tab['chrom']).astype('str')
	calib_result = {}
	for i, chrom in enumerate(chrom_list):
		temp = tab[tab['chrom'] == chrom]
		size = chrom_start_end[i, 1] - chrom_start_end[i, 0]
		vec = np.zeros(size)
		indice = (np.array(temp['bin'] / res)).astype('int')
		v = np.array(temp['value'])
		v[v == -1] = np.nan
		vec[indice] = v
		calib_result[chrom] = vec
	np.save(os.path.join(temp_dir, "calib.npy"), calib_result, allow_pickle=True)


def start_call_compartment(output):
	p_list = []
	pool = ProcessPoolExecutor(max_workers=10)
	
	if ".hdf5" not in output:
		output += ".hdf5"
	with h5py.File(os.path.join(temp_dir, output), "w") as output_f:
		result = {}
		for chrom in chrom_list:
			# real_bulk_compartment, bulk_compartment, temp_compartment_list, temp_compartment_zscore, temp_compartment_quantile, chrom, use_rows, size = process_one_chrom(chrom)
			real_bulk_compartment, bulk_compartment, chrom, use_rows, size,origin_bulk_list,temp_compartment_list = process_one_chrom(chrom)
		# 	p_list.append(pool.submit(process_one_chrom, chrom))
		# 
		# 
		# for p in as_completed(p_list):
		# 	real_bulk_compartment, bulk_compartment, temp_compartment_list, temp_compartment_zscore, temp_compartment_quantile, chrom, use_rows, size = p.result()
			result[chrom] = [real_bulk_compartment, bulk_compartment, use_rows, size,origin_bulk_list,temp_compartment_list]
			
		bin_chrom_list = []
		bin_start_list = []
		bin_end_list = []
		bulk_cp_all = []
		real_bulk_cp_all = []
		sc_cp_all = []
		sc_cp_raw = []
		sc_cp_zscore = []
		sc_cp_bulk = []
		#####################################整个数据集的bulk值##########################################################
		# grp = output_f.create_group('compartment')
		# bin = grp.create_group('bin')
		#############################################################################################################
		
		for chrom in chrom_list:
			real_bulk_compartment, bulk_compartment, use_rows, size,origin_bulk_list ,temp_compartment_list= result[chrom]
			# print (use_rows)
			# print(chrom)
			length = size
			bin_chrom_list += [chrom] * len(use_rows)
			bin_start_list.append((np.arange(length) * res).astype('int')[use_rows])
			bin_end_list.append(((np.arange(length) + 1) * res).astype('int')[use_rows])
			bulk_cp_all.append(bulk_compartment)
			real_bulk_cp_all.append(real_bulk_compartment)
			# sc_cp_all.append(temp_compartment_quantile)
			
			# print("temp_compartment_list shap to append:",temp_compartment_list.shape)
			sc_cp_raw.append(temp_compartment_list)
			# print("sc_cp_raw shape",(np.array(sc_cp_raw)).shape)
			sc_cp_bulk.append(origin_bulk_list)
#####################################整个数据集的bulk值########################################################			
		# bin.create_dataset('chrom', data=[l.encode('utf8') for l in bin_chrom_list],
		#                    dtype=h5py.special_dtype(vlen=str))
		# bin.create_dataset('start', data=np.concatenate(bin_start_list))
		# bin.create_dataset('end', data=np.concatenate(bin_end_list))
		# # print("bulk_cp_all:",np.array(bulk_cp_all).shape)
		# bulk_cp_all = np.concatenate(bulk_cp_all, axis=0)
		# # print("bulk_cp_all shape:",np.array(bulk_cp_all).shape)
		# grp.create_dataset("bulk", data=bulk_cp_all)
		
		# if real_bulk_compartment is not None:
		# 	real_bulk_cp_all = np.concatenate(real_bulk_cp_all, axis=0)
		# 	grp.create_dataset("real_bulk", data=real_bulk_cp_all)
#############################################################################################################		
		# sc_cp_all = np.concatenate(sc_cp_all, axis=-1)
		sc_cp_raw = np.concatenate(sc_cp_raw, axis=-2)
		# print('sc_cp_raw:',(np.array(sc_cp_raw)).shape)
		# sc_cp_zscore = np.concatenate(sc_cp_zscore, axis=-1)
		sc_cp_bulk=np.concatenate(sc_cp_bulk, axis=0)
		# print('sc_cp_bulk:',(np.array(sc_cp_bulk)).shape)
		
		# for cell in range(len(sc_cp_all)):
		# 	grp.create_dataset("cell_%d" % cell, data=sc_cp_all[cell])
		
		grp = output_f.create_group('compartment_raw')
		bin = grp.create_group('bin')
		bin.create_dataset('chrom', data=[l.encode('utf8') for l in bin_chrom_list],
		                   dtype=h5py.special_dtype(vlen=str))
		bin.create_dataset('start', data=np.concatenate(bin_start_list))
		bin.create_dataset('end', data=np.concatenate(bin_end_list))
		for cell in range(len(sc_cp_raw)):
			grp.create_dataset("cell_%d" % cell, data=sc_cp_raw[cell])
		
		# grp = output_f.create_group('compartment_zscore')
		# bin = grp.create_group('bin')
		# bin.create_dataset('chrom', data=[l.encode('utf8') for l in bin_chrom_list],
		#                    dtype=h5py.special_dtype(vlen=str))
		# bin.create_dataset('start', data=np.concatenate(bin_start_list))
		# bin.create_dataset('end', data=np.concatenate(bin_end_list))
		# for cell in range(len(sc_cp_all)):
		# 	grp.create_dataset("cell_%d" % cell, data=sc_cp_zscore[cell])
		grp = output_f.create_group('compartment_bulk')
		bin = grp.create_group('bin')
		bin.create_dataset('chrom', data=[l.encode('utf8') for l in bin_chrom_list],
		                   dtype=h5py.special_dtype(vlen=str))
		bin.create_dataset('start', data=np.concatenate(bin_start_list))
		bin.create_dataset('end', data=np.concatenate(bin_end_list))
		cell_type_info=pickle.load(open(os.path.join(data_dir, "label_info.pickle"), "rb"))
		cell_type=cell_type_info['cell_type']
		i=0
		for j in list(set(cell_type)):
			grp.create_dataset(j, data=sc_cp_bulk[:,:,i])
			i+=1  

			

	output_f.close()
	pool.shutdown(wait=True)

def test_compartment(matrix, return_PCA=False, model=None, expected = None):
	contact = matrix
	# np.fill_diagonal(contact, np.max(contact))
	# contact = KRnormalize(matrix)
	# contact[np.isnan(contact)] = 0.0
	contact = sqrt_norm(matrix)
	contact = oe(contact, expected)
	np.fill_diagonal(contact, 1)
	with warnings.catch_warnings():
		warnings.filterwarnings(
			"ignore", category=PearsonRConstantInputWarning
		)
		contact = pearson(contact)
	np.fill_diagonal(contact, 1)
	contact[np.isnan(contact)] = 0.0
	if model is not None:
		y = model.transform(contact)
	else:
		# pca = PCA(n_components=3)       #保留的pca的列数
		pca = PCA(n_components=1)
		y = pca.fit_transform(contact)
	if return_PCA:
		return y, pca
	else:
		return y


# args = parse_args()
# print (args)
# config = get_config('/home/python/higashi/cellcycle/1M/data/config_cellcycle.JSON')
output = 'sc_and_bulk_cortex50k_hic'
config=get_config('/home/python/higashi/dataset_hic/dataset2/cortex50k/config_cortex50k.json')
res = config['resolution']
data_dir = config['data_dir']
temp_dir = config['temp_dir']
raw_dir = os.path.join(temp_dir, "raw")
if 'cytoband_path' in config:
	cytoband_path = config['cytoband_path']
else:
	cytoband_path = None
neighbor_num = config['neighbor_num']
embedding_name = config['embedding_name']

# if args.calib:
# 	process_calib_file(args.calib_file)
start_call_compartment(output)

(3910, 3910)


100%|██████████| 1075/1075 [1:04:29<00:00,  3.60s/it]


chr1 finished
(3643, 3643)


100%|██████████| 1075/1075 [53:41<00:00,  3.00s/it]


chr2 finished
(3201, 3201)


 59%|█████▉    | 636/1075 [23:23<15:46,  2.16s/it]